In [1]:
import json
from plaid import Client
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pygsheets as pg
gc = pg.authorize(service_file='interview-278411-6a33e71e2568.json')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rU6aficngbrgH-U-lFMhlg5evP0ezkBXwYmIs1rr34k/edit?usp=sharing')

In [3]:
client = Client(
    client_id='5ecc0aaa1aa68500131ccdbe',
    secret='b53550399649cf9a7146a708aefcd6',
    public_key='ad7972d0977a792409762b3f01e061',
    environment='sandbox',
    api_version='2019-05-29'
)
access_token = 'access-sandbox-1847d97d-8392-40fc-a024-53eb2f204b72'
response = client.Transactions.get(access_token,
                                   start_date='2020-01-01',
                                   end_date='2020-04-30', count=500)
transactions = response['transactions']


In [4]:
df = pd.DataFrame(transactions)
sh[1].set_dataframe(df, start = (1,1))

In [5]:
for i in range(len(transactions)):
    if transactions[i]['category'][0] != 'Payment':
        transactions[i]['subcategory'] = transactions[i]['category'][1]
    else:
        transactions[i]['subcategory'] = None
    transactions[i]['category'] = transactions[i]['category'][0]
data = pd.DataFrame(transactions)
data = data[['date', 'category', 'subcategory','amount']]
data['date'] = data['date'].apply(lambda x: x[:-3])
data

,date,category,subcategory,amount
0,2020-04,Travel,Airlines and Aviation Services,500.00
1,2020-04,Travel,Taxi,6.33
2,2020-04,Food and Drink,Restaurants,500.00
3,2020-04,Payment,None,2078.50
4,2020-04,Food and Drink,Restaurants,500.00
...,...,...,...,...
60,2020-01,Food and Drink,Restaurants,12.00
61,2020-01,Food and Drink,Restaurants,4.33
62,2020-01,Food and Drink,Restaurants,89.40
63,2020-01,Transfer,Credit,-4.22


In [6]:
data_income = data.loc[data['amount'] < 0]
data_income['amount'] = data_income['amount'] * -1
data_expenses = data.loc[data['amount'] > 0]
data_expenses['amount'] = data_expenses['amount'] * -1

In [7]:
data_income = data_income.groupby(['date','category', 'subcategory']).sum()
data_income

amount
date    category subcategory                           
2020-01 Transfer Credit                            4.22
        Travel   Airlines and Aviation Services  500.00
2020-02 Transfer Credit                            4.22
        Travel   Airlines and Aviation Services  500.00
2020-03 Transfer Credit                            4.22
        Travel   Airlines and Aviation Services  500.00
2020-04 Transfer Credit                            4.22
        Travel   Airlines and Aviation Services  500.00

In [8]:
data_income['total_category'] = data_income.groupby(['date','category']).sum()
data_income

amount  total_category
date    category subcategory                                           
2020-01 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-02 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-03 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-04 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00

In [9]:
data_income['total_category'] = data_income.groupby(['date','category']).sum()
data_income

amount  total_category
date    category subcategory                                           
2020-01 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-02 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-03 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00
2020-04 Transfer Credit                            4.22            4.22
        Travel   Airlines and Aviation Services  500.00          500.00

Тотал по подкатегориям

In [10]:
data_income.reset_index().groupby('subcategory').sum()

,amount,total_category
subcategory,,
Airlines and Aviation Services,2000.00,2000.00
Credit,16.88,16.88


Тотал по месяцам

In [11]:
data_income.reset_index().groupby('date').sum()

,amount,total_category
date,,
2020-01,504.22,504.22
2020-02,504.22,504.22
2020-03,504.22,504.22
2020-04,504.22,504.22


In [12]:
data_expenses = data_expenses.groupby(['date','category', 'subcategory']).sum()
data_expenses

amount
date    category       subcategory                            
2020-01 Food and Drink Restaurants                    -1105.73
        Recreation     Gyms and Fitness Centers         -78.50
        Shops          Sporting Goods                  -500.00
        Transfer       Debit                          -5850.00
                       Deposit                        -1000.00
        Travel         Airlines and Aviation Services -1000.00
                       Taxi                             -11.73
2020-02 Food and Drink Restaurants                    -1105.73
        Recreation     Gyms and Fitness Centers         -78.50
        Shops          Sporting Goods                  -500.00
        Transfer       Debit                          -5850.00
                       Deposit                        -1000.00
        Travel         Taxi                             -11.73
2020-03 Food and Drink Restaurants                    -1105.73
        Recreation     Gyms and Fitness Centers         -78.50
        Shops          Sporting Goods                  -500.00
        Transfer       Debit                          -5850.00
                       Deposit                        -1000.00
        Travel         Airlines and Aviation Services -1000.00
                       Taxi                             -11.73
2020-04 Food and Drink Restaurants                    -1105.73
        Recreation     Gyms and Fitness Centers         -78.50
        Shops          Sporting Goods                  -500.00
        Transfer       Debit                          -5850.00
                       Deposit                        -1000.00
        Travel         Airlines and Aviation Services  -500.00
                       Taxi                             -11.73

In [13]:
data_expenses['total_category'] = data_expenses.groupby(['date','category']).sum()
data_expenses

amount  total_category
date    category       subcategory                                            
2020-01 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services -1000.00        -1011.73
                       Taxi                             -11.73        -1011.73
2020-02 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Taxi                             -11.73          -11.73
2020-03 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services -1000.00        -1011.73
                       Taxi                             -11.73        -1011.73
2020-04 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services  -500.00         -511.73
                       Taxi                             -11.73         -511.73

In [14]:
data_expenses.groupby(['subcategory']).sum()
data_expenses.groupby(['category']).sum()
data_expenses

amount  total_category
date    category       subcategory                                            
2020-01 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services -1000.00        -1011.73
                       Taxi                             -11.73        -1011.73
2020-02 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Taxi                             -11.73          -11.73
2020-03 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services -1000.00        -1011.73
                       Taxi                             -11.73        -1011.73
2020-04 Food and Drink Restaurants                    -1105.73        -1105.73
        Recreation     Gyms and Fitness Centers         -78.50          -78.50
        Shops          Sporting Goods                  -500.00         -500.00
        Transfer       Debit                          -5850.00        -6850.00
                       Deposit                        -1000.00        -6850.00
        Travel         Airlines and Aviation Services  -500.00         -511.73
                       Taxi                             -11.73         -511.73

Тотал по подкатегориям

In [15]:
data_expenses.reset_index().groupby('subcategory').sum()

,amount,total_category
subcategory,,
Airlines and Aviation Services,-2500.00,-2535.19
Debit,-23400.00,-27400.00
Deposit,-4000.00,-27400.00
Gyms and Fitness Centers,-314.00,-314.00
Restaurants,-4422.92,-4422.92
Sporting Goods,-2000.00,-2000.00
Taxi,-46.92,-2546.92


Тотал по месяцам

In [16]:
data_expenses.reset_index().groupby('date').sum()

,amount,total_category
date,,
2020-01,-9545.96,-17407.69
2020-02,-8545.96,-15395.96
2020-03,-9545.96,-17407.69
2020-04,-9045.96,-16407.69


In [17]:
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rU6aficngbrgH-U-lFMhlg5evP0ezkBXwYmIs1rr34k/edit?usp=sharing')
sh[0].set_dataframe(data_expenses.reset_index(), start = (1,1))
sh[0].set_dataframe(data_income.reset_index(), start = (30,1))